# MAXIMAL COVERAGE ALGORITHM

## Libraries

In [1]:
import pandas as pd

## Constants

In [2]:
input_folder_path = "/Users/Jovan/Desktop/MDA-Project/Data"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]
cities = ["Oostende"]

COVERAGE_RADIUS = 150

## Comparisons

In [8]:
df = pd.read_csv("/Users/Jovan/Desktop/Oostende_cost_matrix.csv", index_col=0)

num_columns = df.shape[1]
mid_index = num_columns // 2
old_aed_columns = df.iloc[:, :mid_index]
new_aed_columns = df.iloc[:, mid_index:]

df['Old_AED_within_150m'] = old_aed_columns.apply(lambda row: (row < 150).any(), axis=1)
df['New_AED_within_150m'] = new_aed_columns.apply(lambda row: (row < 150).any(), axis=1)

percentage_old_aed = df['Old_AED_within_150m'].mean() * 100
percentage_new_aed = df['New_AED_within_150m'].mean() * 100

# Print the results
print(df[['Old_AED_within_150m', 'New_AED_within_150m']])
print(f"Percentage of cardiac arrests with an AED within 150m (old placements): {percentage_old_aed:.2f}%")
print(f"Percentage of cardiac arrests with an AED within 150m (new placements): {percentage_new_aed:.2f}%")

                     Old_AED_within_150m  New_AED_within_150m
51.232792, 2.944167                False                False
51.228196, 2.910508                False                 True
51.229511, 2.913998                False                 True
51.234277, 2.915419                False                False
51.234876, 2.916587                False                False
...                                  ...                  ...
51.231357, 2.945571                False                False
51.211095, 2.878669                False                False
51.230009, 2.915204                 True                 True
51.219038, 2.909445                False                False
51.216481, 2.890583                False                False

[100 rows x 2 columns]
Percentage of cardiac arrests with an AED within 150m (old placements): 17.00%
Percentage of cardiac arrests with an AED within 150m (new placements): 32.00%
